In [1]:
from neuron import h
import os
import matplotlib.pyplot as plt

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))




#First compile our modfiles.
if os.system('nrnivmodl modfiles/') == 0:
    print("Compilation Succeeded.")
else:
    print("Compilation Failed.")

Compilation Succeeded.


In [2]:
h.load_file('stdrun.hoc')

#Define simumulation parameters.
SIMRUNTIME = 1500
DELAY      = 400
INJTIME    = 500
I_INJ      = 0.2

h.tstop    = SIMRUNTIME
h.v_init   = -75

#Now set up vectors to record the membrane potential and the time.
potential = h.Vector()
time      = h.Vector()

In [5]:
import numpy as np

tonic_soma = h.Section(name = 'tonic_soma')

tonic_iclamp = h.IClamp(tonic_soma(0.5))
tonic_iclamp.dur   = INJTIME
tonic_iclamp.amp   = I_INJ
tonic_iclamp.delay = DELAY

#We will make a super simple one compartement tonic spiking cell.
tonic_soma.nseg = 1
tonic_soma.L    = 25
tonic_soma.diam = 24.75

#Now insert the two segregated modules into the cell.
#First insert the passive module.
tonic_soma.insert('leakCA3')
tonic_soma.insert('hdCA3')

#Now insert the spiking module.
tonic_soma.insert('natCA3')
tonic_soma.insert('kdrCA3')

#record time and membrane potential for tonic cell.
#Record the membrane variables.
time.record(h._ref_t) 
potential.record(tonic_soma(0.5)._ref_v)


def run_tonic_model(amp, gleak, gna, gkdr, gh, tau_m, tau_h, tau_n, tau_l, vh_m, vh_h, vh_n, vh_l):

    tonic_iclamp.amp = amp

    tonic_soma.glbar_leakCA3   = gleak #1.53*3e-5
    tonic_soma.gbar_natCA3     = gna
    tonic_soma.mtfactor_natCA3 = tau_m
    tonic_soma.htfactor_natCA3 = tau_h
    tonic_soma.vhalfm_natCA3   = vh_m
    tonic_soma.vhalfh_natCA3   = vh_h
    tonic_soma.gbar_kdrCA3     = gkdr
    tonic_soma.ntfactor_kdrCA3 = tau_n
    tonic_soma.vhalfn_kdrCA3   = vh_n
    tonic_soma.ghdbar_hdCA3    = gh
    tonic_soma.ltfactor_hdCA3  = tau_l
    tonic_soma.vhalfl_hdCA3    = vh_l
    
    h.run()

#Set the default parameters for the simulations.

default_tonic = [I_INJ, 1.53*3e-5, 0.065, 0.065, 1.0e-5, 
                        1.0      , 1.0  , 1.0  , 1.0   ,
                        -35.5    , -40  , 0.0  , -75.0  ]

run_tonic_model(default_tonic[0], default_tonic[1], default_tonic[2],  default_tonic[3],  default_tonic[4],
                                  default_tonic[5], default_tonic[6],  default_tonic[7],  default_tonic[8],
                                  default_tonic[9], default_tonic[10], default_tonic[10], default_tonic[12] )


run complete
